The PyIsaricBasics package exists to introduce the user to the ISARIC dataset and provide some basic data exploration and functionality for each of the domains. It consists of some functions to read and write data from the raw .csv files as well as a Class that loads a domain and provides some high level methods that letus take a look at some of the data. The Domain Class stores an object called a Pandas DataFrame as its underlying datastructure, the Pandas library (https://pandas.pydata.org) is an open source library for data analysis, visualisation and manipulation inside Python. 

The PyIsaricBasics provides functionality so that you can do some basic data exploration without having to delve into the Pandas library. However it also provides methods that allow you to access the underlying DataFrame if you need to! The Pandas documentation is extensive which means for more complicated analysis you can use this package to access and select the data you need before utilising the Pandas ecosystem for downstream analysis. 

We first set some global variables. DATA_DIRECTORY is a path to the directory where the raw ISARIC .csv's are contained. While DATABASE_FILE is what we want the sqlite database to be named. 

In [1]:
DATA_DIRECTORY = "tests/Data"
DATABASE_FILE = 'test_db.sqlite'
import time 
start_time = time.time

## PLACEHOLDER  - need to change to pip import 

We now import the Domain Class and some useful functions from the pyIsaricBasics package. 

In [2]:
from src.pyBasics.domain import Domain
from src.pyBasics.functions import *

The first step in our data exploration / analysis is to convert all of our raw .csv's to a sqlite database. This is useful for browsing with the application DB Browser (https://sqlitebrowser.org). 

Unfortunately reading and writing full sqlite tables into memory as a dataframe is not particularly efficient in Python 3. However this function also creates auxiliary .pickle files that contain a serialised version of pandas DataFrame objects - loading these files is much more efficient. Generating the inital database can take some time (approximately 20mins on a laptop), we suggest you let this run and then have a read through the pyIsaricBasics documentation: 

In [3]:
# csv_to_sqlite(DATA_DIRECTORY, DATABASE_FILE)

## Load an example domain -> lets use the SA domain
We use a custom class "Domain()" to create a wrapper around a Pandas dataframe, this lets us define methods for basic data analysis and exploration that are common across all domains. We also do some basic preprocessing within this method for some domains. The 'num_rows' argument is an optional argument that lets us load a subset of the full data. This is useful as some of the domains (e.g. SA which we use in this tutorial) have > 35 million rows! With such a large dataset even simple numeric operations can take a long time. 


In [4]:
toc = time.time()
SA = Domain("SA", DATA_DIRECTORY)
tic = time.time()
print(tic - toc)

55.61565089225769


We can now use some of the common methods of the Domain class. 
It's useful to know what columns is contained within our DataFrame, which we can do as follows: 

In [5]:
SA.columns()

['STUDYID', 'DOMAIN', 'USUBJID', 'SASEQ', 'SATERM', 'SAMODIFY', 'SACAT', 'SASCAT', 'SAPRESP', 'SAOCCUR', 'SASTAT', 'SAREASND', 'SALOC', 'SADY', 'SASTDY', 'SATPT', 'SATPTREF', 'SASTRF', 'SAEVLINT', 'SAEVINTX', 'SARPOC', 'status']


The Domain.columns() function print's a list of the columns in our current domain. 

All the columns in UPPERCASE are unaltered from the original SA csv file. We also have one extra column 'status', which converts the outcomes from ISARIC / STDM format into a simple "Y", "N" or "U". In general it is good practice to stick to lower case for the names of any columns that we create or derive ourselves. 

Some important columns from the original ISARIC data are:
    
    xxTERM - Contains the verbatim non-standardised wording of an event
    xxOCCUR - Helps to determine whether an event occured or not
    xxPREPSP - a value of 'y' in this column indicates that the event was prespecified on the CRF, while 'n' or missing indicates a spontaneous (or free-text entry)
    xxSTDY - Gives the day of an event (relative to admission day) 
    
The 'status' variable indicates whether an event occurred based on the combination of values in xxPRESP and xxOCCUR

Now we know what the columns in our table are, it could be useful to look at the missingness in different columns:

In [6]:
SA.table_missingness()

Total number of rows 31923533
STUDYID            0
DOMAIN             0
USUBJID            0
SASEQ              0
SATERM            13
SAMODIFY      255150
SACAT         110859
SASCAT      19858160
SAPRESP       387793
SAOCCUR       488399
SASTAT      31840251
SAREASND    31840252
SALOC       31918377
SADY        10083301
SASTDY      31679830
SATPT       30634974
SATPTREF    30634974
SASTRF      31918482
SAEVLINT    31201067
SAEVINTX    20720309
SARPOC      31920431
status             0
dtype: int64


This method prints out the number of rows in each column that have missing values, as well as the total number of rows of the column. 

As you can see there is a large number of columns with high missingness -> we can exclude some of these columns from our dataframe, which has the benefit of freeing up memory and computation time 

In [7]:
SA.exclude_columns(['SASCAT', "SASTAT", "SAREASND", "SALOC", "SATPT", "SATPTREF", "SASTRF", "SAEVINTX", "SARPOC"])

Now if we want to take a closer look at some of the variables in each column we can do so with the following method: 

In [8]:
SA.column_events("SAMODIFY")

['OTHER (NOT SPECIFIED)' 'TUBERCULOSIS' 'ARDS' 'PULMONARY EMBOLISM OR DVT'
 'MULTISYSTEM INFLAMMATORY SYNDROME'
 'DISSEMINATED INTRAVASCULAR COAGULATION' 'NEUROLOGICAL COMPLICATION'
 'CHRONIC PULMONARY DISEASE (NOT ASTHMA)' 'HYPERTENSION'
 'DIABETES MELLITUS - TYPE NOT SPECIFIED' 'NOSOCOMIAL SEPSIS' 'OBESITY'
 'SMOKING' 'ACUTE KIDNEY INJURY'
 'CHRONIC CARDIAC DISEASE (NOT HYPERTENSION)' 'MALIGNANT NEOPLASM'
 'ASTHMA' 'HIV' 'OTHER COMPLICATION (NOT SPECIFIED)'
 'CHRONIC KIDNEY DISEASE' 'SHOCK' 'CLINICALLY-DIAGNOSED COVID-19'
 'WEIGHT LOSS' nan 'PULMONARY EMBOLISM' 'PALPITATIONS'
 'COUGH - NON-PRODUCTIVE' 'CONJUNCTIVITIS' 'SEIZURES' 'DIZZINESS'
 'DEEP VEIN THROMBOSIS' 'KIDNEY DISEASE' 'SKIN RASH' 'PARAESTHESIA'
 'SHORTNESS OF BREATH' 'MUSCLE WEAKNESS' 'PROBLEMS WITH BALANCE'
 'PROBLEMS SLEEPING' 'ERECTILE DYSFUNCTION' 'ABDOMINAL PAIN'
 'COUGH - PRODUCTIVE' 'CHEST PAIN' 'PROBLEMS SWALLOWING OR CHEWING'
 'DIARRHOEA' 'FEVER/HISTORY OF FEVER' 'LOSS OF SENSATION'
 'CHANGES IN MENSTRUATION' 'C

We can now look at the table missingness while filtering on a specific variable. For example if we are interested in 'HYPERTENSION':


In [9]:
SA.table_missingness("SAMODIFY", "HYPERTENSION")

Total number of rows 632964
None
STUDYID          0
DOMAIN           0
USUBJID          0
SASEQ            0
SATERM           0
SAMODIFY         0
SACAT            1
SAPRESP       5708
SAOCCUR       8159
SADY        394878
SASTDY      632964
SAEVLINT    632947
status           0
dtype: int64


Now lets take a closer look at the filtered DataFrame: 

In [10]:
SA.select_variable_from_column("SAMODIFY", "HYPERTENSION")

STUDYID DOMAIN              USUBJID  SASEQ               SATERM      SAMODIFY            SACAT SAPRESP SAOCCUR   SADY  SASTDY SAEVLINT status
10        CVZXZMV     SA       CVZXZMV_100126     19         HYPERTENSION  HYPERTENSION  MEDICAL HISTORY       Y       U    NaN     NaN      NaN      U
24        CVZXZMV     SA       CVZXZMV_100267     16         HYPERTENSION  HYPERTENSION  MEDICAL HISTORY       Y       N    NaN     NaN      NaN      N
53        CVZXZMV     SA       CVZXZMV_100472     19         HYPERTENSION  HYPERTENSION  MEDICAL HISTORY       Y       N    NaN     NaN      NaN      N
70        CVZXZMV     SA       CVZXZMV_100581     13         HYPERTENSION  HYPERTENSION  MEDICAL HISTORY       Y       N    NaN     NaN      NaN      N
74        CVZXZMV     SA       CVZXZMV_100643     20         HYPERTENSION  HYPERTENSION  MEDICAL HISTORY       Y       Y    NaN     NaN      NaN      Y
...           ...    ...                  ...    ...                  ...           ...              ...     ...     ...    ...     ...      ...    ...
31922841  CVSURVY     SA   CVSURVY_00840_1867     31  HIGH BLOOD PRESSURE  HYPERTENSION  MEDICAL HISTORY       Y       N  183.0     NaN      NaN      N
31922901  CVSURVY     SA   CVSURVY_00840_1345     32  HIGH BLOOD PRESSURE  HYPERTENSION  MEDICAL HISTORY       Y       N  183.0     NaN      NaN      N
31923085  CVSURVY     SA  CVSURVY_00825-TH115     34  HIGH BLOOD PRESSURE  HYPERTENSION  MEDICAL HISTORY       Y       Y  186.0     NaN      NaN      Y
31923408  CVSURVY     SA   CVSURVY_00840_2348     32  HIGH BLOOD PRESSURE  HYPERTENSION  MEDICAL HISTORY       Y       N  182.0     NaN      NaN      N
31923469  CVSURVY     SA   CVSURVY_00840_3618     33  HIGH BLOOD PRESSURE  HYPERTENSION  MEDICAL HISTORY       Y       N   96.0     NaN      NaN      N

[632964 rows x 13 columns]

Its worthwhile noting that this method returns a Pandas DataFrame, so we can use anything contained in the Pandas library to further filter this dataframe. For instance if we create a list of columns that we're interested in we can use this to only display these columns: 

In [11]:
cols_of_interest = ["USUBJID", "SASTDY", "SAMODIFY", "SAPRESP", "SAOCCUR", 'status']
SA.select_variable_from_column("SAMODIFY", "HYPERTENSION")[cols_of_interest]

USUBJID  SASTDY      SAMODIFY SAPRESP SAOCCUR status
10             CVZXZMV_100126     NaN  HYPERTENSION       Y       U      U
24             CVZXZMV_100267     NaN  HYPERTENSION       Y       N      N
53             CVZXZMV_100472     NaN  HYPERTENSION       Y       N      N
70             CVZXZMV_100581     NaN  HYPERTENSION       Y       N      N
74             CVZXZMV_100643     NaN  HYPERTENSION       Y       Y      Y
...                       ...     ...           ...     ...     ...    ...
31922841   CVSURVY_00840_1867     NaN  HYPERTENSION       Y       N      N
31922901   CVSURVY_00840_1345     NaN  HYPERTENSION       Y       N      N
31923085  CVSURVY_00825-TH115     NaN  HYPERTENSION       Y       Y      Y
31923408   CVSURVY_00840_2348     NaN  HYPERTENSION       Y       N      N
31923469   CVSURVY_00840_3618     NaN  HYPERTENSION       Y       N      N

[632964 rows x 6 columns]

When we select only these columns the relationship between SAPRESP, SAOCCUR and status becomes a little more evident too. 

Another useful piece of functionality in the Domain class lets us print off a summary of the values that occur in each column: 

In [12]:
SA.column_summary("SAMODIFY")

DIABETES MELLITUS - TYPE NOT SPECIFIED              1050801
OBESITY                                             1031524
TUBERCULOSIS                                         835876
CHRONIC CARDIAC DISEASE (NOT HYPERTENSION)           823236
MALIGNANT NEOPLASM                                   814181
HIV                                                  663760
CHRONIC KIDNEY DISEASE                               662347
CHRONIC PULMONARY DISEASE (NOT ASTHMA)               661730
ASTHMA                                               659295
SMOKING                                              644254
ACUTE KIDNEY INJURY                                  643887
HYPERTENSION                                         632964
ARDS                                                 632289
OTHER COMPLICATION (NOT SPECIFIED)                   588755
MUSCLE ACHES/JOINT PAIN                              568081
NEUROLOGICAL COMPLICATION                            549145
SHORTNESS OF BREATH                     

We can also use this method to see the proportions of each variable as well:

In [13]:
SA.column_summary("SAMODIFY", proportions = True)

DIABETES MELLITUS - TYPE NOT SPECIFIED              3.318139e-02
OBESITY                                             3.257268e-02
TUBERCULOSIS                                        2.639465e-02
CHRONIC CARDIAC DISEASE (NOT HYPERTENSION)          2.599552e-02
MALIGNANT NEOPLASM                                  2.570959e-02
HIV                                                 2.095971e-02
CHRONIC KIDNEY DISEASE                              2.091509e-02
CHRONIC PULMONARY DISEASE (NOT ASTHMA)              2.089560e-02
ASTHMA                                              2.081871e-02
SMOKING                                             2.034376e-02
ACUTE KIDNEY INJURY                                 2.033217e-02
HYPERTENSION                                        1.998725e-02
ARDS                                                1.996594e-02
OTHER COMPLICATION (NOT SPECIFIED)                  1.859126e-02
MUSCLE ACHES/JOINT PAIN                             1.793843e-02
NEUROLOGICAL COMPLICATION

However this just gives us the counts / proportions of events that are recorded without any information on the status of the event (e.g Y, N or U). Luckily we can use the status keyword to get this information. 

In [14]:
SA.column_summary("SAMODIFY", status = True)

ABDOMINAL PAIN__N                                      228210
ABDOMINAL PAIN__U                                       32344
ABDOMINAL PAIN__Y                                       21631
ACUTE CARDIAC INJURY__N                                   532
ACUTE CARDIAC INJURY__U                                    88
ACUTE CARDIAC INJURY__Y                                    60
ACUTE GASTROENTERITIS__U                                   40
ACUTE GASTROENTERITIS__Y                                   37
ACUTE KIDNEY INJURY__N                                 373933
ACUTE KIDNEY INJURY__U                                 231256
ACUTE KIDNEY INJURY__Y                                  38698
ALTERED CONSCIOUSNESS/CONFUSION__N                     207548
ALTERED CONSCIOUSNESS/CONFUSION__U                      25675
ALTERED CONSCIOUSNESS/CONFUSION__Y                      48003
ANAEMIA__N                                             203362
ANAEMIA__U                                               9386
ANAEMIA_

We can also use the Domain class to search for 'free text' variables. For most variables in the ISARIC dataset, the xxMODIFY column contains a standardised event name. However for some spontaneously recorded events this might not be the case. In some instances it can be worthwhile checking these entries... 

In this example we are going to search the SA domain for some terms that might be relevant to Kidney Stones (for which there is no standardised variable in the 'SAMODIFY' column. We use the domain.free_text_search() method. We can enter any terms we wish to search for as strings separated by commas. This method then searches for any of these terms in the relevant column and returns a dataframe with the result. 

In [15]:
stones_frame = SA.free_text_search("kidney stones", "nephrolithiasis", "renal calculi")

Free text entries containing any of kidney stones or nephrolithiasis or renal calculi were found in 271 rows


In [16]:
stones_frame

STUDYID DOMAIN             USUBJID  SASEQ                               SATERM SAMODIFY            SACAT SAPRESP SAOCCUR  SADY  SASTDY SAEVLINT status
839322    CVZXZMV     SA      CVZXZMV_260259     16                        RENAL CALCULI      NaN  MEDICAL HISTORY     NaN     NaN   NaN     NaN      NaN      Y
1338607   CVZXZMV     SA      CVZXZMV_384798      6                        KIDNEY STONES      NaN    COMPLICATIONS     NaN     NaN   NaN     NaN      NaN      Y
3655180   CVZXZMV     SA      CVZXZMV_333242     23  KIDNEY STONES AND COVID-19 POSITIVE      NaN  MEDICAL HISTORY     NaN     NaN   NaN     NaN      NaN      Y
4881755   CVZXZMV     SA       CVZXZMV_66860     22                        KIDNEY STONES      NaN  MEDICAL HISTORY     NaN     NaN   NaN     NaN      NaN      Y
9178547   CVZXZMV     SA      CVZXZMV_438607     10           ADMITTED FOR KIDNEY STONES      NaN  MEDICAL HISTORY     NaN     NaN   NaN     NaN      NaN      Y
...           ...    ...                 ...    ...                                  ...      ...              ...     ...     ...   ...     ...      ...    ...
31446318  CVCCPUK     SA  CVCCPUK_7A4BV-0880     31              RECURRENT KIDNEY STONES      NaN  MEDICAL HISTORY     NaN     NaN   1.0     NaN      NaN      Y
31534376  CVCCPUK     SA  CVCCPUK_RLT01-0706      7                      NEPHROLITHIASIS      NaN  MEDICAL HISTORY     NaN     NaN   1.0     NaN      NaN      Y
31653094  CVCCPUK     SA  CVCCPUK_RBA11-1188     18                        RENAL CALCULI      NaN  MEDICAL HISTORY     NaN     NaN   1.0     NaN      NaN      Y
31840748  CVCCPUK     SA  CVCCPUK_RDDH0-1157     17                        RENAL CALCULI      NaN    COMPLICATIONS     NaN     NaN  13.0     NaN      NaN      Y
31868910  CVCCPUK     SA  CVCCPUK_REM21-1319     57              RECURRENT KIDNEY STONES      NaN  MEDICAL HISTORY     NaN     NaN   1.0     NaN      NaN      Y

[271 rows x 13 columns]

So in our reduced data frame (only 1,000,000 rows) we found one free text entry that is relevant for Kidney stones. We would expect to see more if we used the full ~35 Million rows. Note that the value of SAPRESP is NaN (missing) as is the value of SAOCCUR. This indicates that the entry was made spontaenously (i.e. not indicated on the CRF) 

Now we should save our modified DataFrame (with the added status variable) back into a sqlite table:

(note this takes quite a while for big domains such as SA and IN) 

In [17]:
SA.save_to_sqlite("SA", DATA_DIRECTORY, DATABASE_FILE )

This overwrites the previously contained sqlite table and also updates the .pickle file to contain the newly created status column

# Vaccination Status Example

Now we have introduced the basic functionality of our package we are going to give an example of using the package to retrieve the vaccination status of patients. 

In this example we need to load the IN domain as this contains information about vaccinations (note we first delete the SA domain from memory to save some space). We also again select a subset of rows from the IN domain just to speed up the example. 


In [18]:
start_time = time.time()
del(SA)

In [19]:
IN = Domain("IN", DATA_DIRECTORY, 2000000)

We then inspect the columns:

In [20]:
IN.columns()

['STUDYID', 'DOMAIN', 'USUBJID', 'SPDEVID', 'INSEQ', 'INREFID', 'INTRT', 'INMODIFY', 'INCAT', 'INSCAT', 'INPRESP', 'INOCCUR', 'INCLAS', 'INCLASCD', 'INSTAT', 'INREASND', 'ININDC', 'INDOSE', 'INDOSTXT', 'INDOSU', 'INDOSFRM', 'INDOSFRQ', 'INDOSTOT', 'INROUTE', 'INDY', 'INSTDY', 'INENDY', 'INDUR', 'INTPT', 'INTPTREF', 'INSTRF', 'INEVLINT', 'INEVINTX', 'INCDSTDY', 'status']


Most of those columns are not relevant to vaccination status so we're going to include only relevant columns

In [21]:
relevant_cols = ['USUBJID', 'INTRT', 'INMODIFY', 'INPRESP', 'INOCCUR', 'INREFID' ,'INSTDY', 'status']
IN.include_columns(relevant_cols)

While there are derived values for COVID-19 vaccination status in the 'INMODIFY' column, they contain different values depending on the type of vaccination received. Instead we are going to search the 'INTRT' column with a variety of free-text search terms to ensure we get as many COVID-19 vaccination events as possible, including those events that do not contain a value in the standardised column. 

In [22]:
covid_vacc = IN.free_text_search("COVID-19 Vaccine", "ASTRAZENECA", "PFIZER", "COVISHIELD",
                                 "SINOVAC", "COVID-19 VACCINATION", "RECEIVED A COVID-19 VACCIN")

Free text entries containing any of COVID-19 Vaccine or ASTRAZENECA or PFIZER or COVISHIELD or SINOVAC or COVID-19 VACCINATION or RECEIVED A COVID-19 VACCIN were found in 43171 rows


So out of our 2,000,000 total events that we found 43,171 events that contained one of the terms above. Lets have a closer look: 

In [23]:
covid_vacc.head(5)

USUBJID                        INTRT              INMODIFY INPRESP INOCCUR INREFID  INSTDY status
26   CVZXZMV_100028  RECEIVED A COVID-19 VACCINE  COVID-19 VACCINATION       Y       U     NaN     NaN      U
68   CVZXZMV_100189  RECEIVED A COVID-19 VACCINE  COVID-19 VACCINATION       Y       U     NaN     NaN      U
98   CVZXZMV_100556  RECEIVED A COVID-19 VACCINE  COVID-19 VACCINATION       Y       U     NaN     NaN      U
157  CVZXZMV_100578  RECEIVED A COVID-19 VACCINE  COVID-19 VACCINATION       Y       N     NaN     NaN      N
215   CVZXZMV_10060  RECEIVED A COVID-19 VACCINE  COVID-19 VACCINATION       Y       U     NaN     NaN      U

Lets look at the unique values for each column using some functionality from Pandas. Each column in a pandas DataFrame is stored as a series. We can access the series directly by using 'df.colname' and then using the .unique() method we can find the unique values contained in that column. 

In [24]:
covid_vacc.INTRT.unique()

array(['RECEIVED A COVID-19 VACCINE', 'COVID-19 VACCINATION',
       'COVID-19 VACCINE PFIZER/BIONTECH',
       'COVID-19 VACCINE JANSSENS (JOHNSON AND JOHNSON)',
       'COVID-19 VACCINE TYPE UNKNOWN', 'PFIZER-BIONTECH',
       'ASTRA ZENECA (COVISHIELD)', 'COVID-19 VACCINE CANSINOBIO'],
      dtype=object)

In [25]:
covid_vacc.INMODIFY.unique()

array(['COVID-19 VACCINATION', 'COVID-19 VACCINE PFIZER-BIONTECH',
       'COVID-19 VACCINE JANSSENS (JOHNSON AND JOHNSON)',
       'COVID-19 VACCINE TYPE UNKNOWN',
       'COVID-19 VACCINE ASTRAZENECA/UNIVERSITY OF OXFORD',
       'COVID-19 VACCINE CANSINBIO'], dtype=object)

In [26]:
covid_vacc.INREFID.unique()

array([nan, 'DOSE 1', 'DOSE 2'], dtype=object)

We can also look at the counts in the 'status' variable in a serious way. 

In [27]:
covid_vacc.status.value_counts()

U    36290
N     5937
Y      944
Name: status, dtype: int64

Great! So now what do we do if we want to save this DataFrame to access it later?

We can use the function df_to_sqlite() which saves a DataFrame into the sqlite database created earlier and as a .pickle which we can load quickly into Python.

In [28]:
df_to_sqlite(covid_vacc, "vacc_status", DATA_DIRECTORY, DATABASE_FILE)

True

In [29]:
finish_time = time.time()
print(finish_time - start_time) 

132.64977717399597
